# Create npz files for sub-images into output folder

In [1]:
# standard
import pandas as pd
import numpy as np
import random
import os

# tf and keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers

# sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# plots
import seaborn as sns
import matplotlib.pyplot as plt

random.seed(2)
%matplotlib inline

In [2]:
import albumentations as A
from PIL import Image
from tqdm import tqdm
import dask.dataframe as dd

In [3]:
# Mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# File paths
BASE_PATH = '/content/drive/MyDrive/MIDS/Data 207/207 Final Project/Models/transformed_data'
splits = ['train', 'val', 'test']

In [5]:
# Function to show image with bounding box
def show_transformed_image(image, boxes, layout=(1, 1, 1)):
    ax = plt.subplot(*layout)

    # Iterate over the individual labels.
    width = image.shape[1]
    height = image.shape[0]
    for i, box in enumerate(boxes):
        # Draw the object bounding box.
        ax.add_patch(patches.Rectangle(
        xy=((box[0] - 0.5 * box[2]) * width,
            (box[1] - 0.5 * box[3]) * height),
        width=box[2] * width,
        height=box[3] * height,
        linewidth=1,
        edgecolor='red',
        facecolor='none'))

    # Show the camera image.
    plt.imshow(image)
    plt.grid(False)
    plt.axis('off')

# Function that returns True/False if pedestrian in subimage
def is_pedestrian_in_sub_image(sub_image_bounds, image_size, labels, min_percentage=0.25):
    left = sub_image_bounds[0]
    right = sub_image_bounds[1]
    top = sub_image_bounds[2]
    bottom = sub_image_bounds[3]

    image_width = image_size[0]
    image_height = image_size[1]
    for l in labels:
        l_left = (l[0] - l[2] / 2) * image_width
        l_right = (l[0] + l[2] / 2) * image_width
        l_top = (l[1] - l[3] / 2) * image_height
        l_bottom = (l[1] + l[3] / 2) * image_height

        cl_left = max(left, l_left)
        cl_right = min(right, l_right)
        cl_top = max(top, l_top)
        cl_bottom = min(bottom, l_bottom)

        if cl_right < cl_left or cl_bottom < cl_top:
            iou = 0
        else:
            iou = get_iou((left, right, top, bottom), (cl_left, cl_right, cl_top, cl_bottom), in_yolo_format=False)

        if iou > min_percentage:
            return True

    return False

# Helper function for is_pedestrian_in_sub_image
def get_iou(bb1, bb2, in_yolo_format=True):
    if in_yolo_format:
        bb1_right = bb1[0] + bb1[2] / 2
        bb1_left = bb1[0] - bb1[2] / 2
        bb1_bottom = bb1[1] + bb1[3] / 2
        bb1_top = bb1[1] - bb1[3] / 2

        bb2_right = bb2[0] + bb2[2] / 2
        bb2_left = bb2[0] - bb2[2] / 2
        bb2_bottom = bb2[1] + bb2[3] / 2
        bb2_top = bb2[1] - bb2[3] / 2
    else:
        bb1_left, bb1_right, bb1_top, bb1_bottom = bb1
        bb2_left, bb2_right, bb2_top, bb2_bottom = bb2

    intersection_width = min(bb1_right, bb2_right) - max(bb1_left, bb2_left)
    intersection_height = min(bb1_bottom, bb2_bottom) - max(bb1_top, bb2_top)

    if intersection_width <= 0 or intersection_height <= 0:
        return 0

    intersection_area = intersection_width * intersection_height

    bb1_area = (bb1_right - bb1_left) * (bb1_bottom - bb1_top)
    bb2_area = (bb2_right - bb2_left) * (bb2_bottom - bb2_top)

    union_area = bb1_area + bb2_area - intersection_area
    iou = intersection_area / union_area
    return iou

# Creates sub images from full image
def create_sub_images(image, labels, sub_image_size=64):
    im = Image.fromarray(image)

    sub_images = []
    sub_image_contains_pedestrian = []
    for y in range(image.shape[0] // sub_image_size):
        for x in range(image.shape[1] // sub_image_size):
            left = x * sub_image_size
            right = left + sub_image_size
            top = y * sub_image_size
            bottom = top + sub_image_size
            sub_image = im.crop((left, top, right, bottom))
            sub_images.append(np.array(sub_image))
            sub_image_contains_pedestrian.append(
                is_pedestrian_in_sub_image((left, right, top, bottom), (image.shape[1], image.shape[0]), labels)
            )
    return sub_images, (image.shape[0] // sub_image_size, image.shape[1] // sub_image_size), sub_image_contains_pedestrian

def reconstruct_sub_images(sub_images, num_sub_images):
    rows = []
    for y in range(num_sub_images[1]):
        row = []
        for x in range(num_sub_images[0]):
            row.append(sub_images[x + y * num_sub_images[1]])
        rows.append(np.concatenate(row, axis=1))
    return np.concatenate(rows, axis=0)

def plot_sub_image_predictions(sub_images, num_sub_images, sub_images_sizes, sub_image_preds, layout=(1, 1, 1)):
    image = reconstruct_sub_images(sub_images, num_sub_images)
    ax = plt.subplot(*layout)

    for y in range(num_sub_images[1]):
        for x in range(num_sub_images[0]):
            i = x + y * num_sub_images[0]
            if not sub_image_preds[i]:
                continue

            ax.add_patch(patches.Rectangle(
                xy=(x * sub_images_sizes[0],
                    y * sub_images_sizes[1]),
                width=sub_images_sizes[0],
                height=sub_images_sizes[1],
                linewidth=1,
                edgecolor='red',
                facecolor='none')
            )
    plt.imshow(image)
    plt.grid(False)
    plt.axis('off')

## Creating Sub-Images
1. Look through each folder (train, val, test).
2. Convert the image + bbox pairs into (64, 64) sub-images.
3. Add labels.
4. Save X and y as SPLIT_data.npz into sub_images folder.

Reminder:
- BASE_PATH = '/content/drive/MyDrive/MIDS/Data 207/207 Final Project/Daniel/transformed_data'
- splits = ['train', 'val', 'test']

In [9]:
# Define output path
OUTPUT_PATH = '/content/drive/MyDrive/MIDS/Data 207/207 Final Project/Models/sub_images'

# Ensure output directory exists
os.makedirs('/content/drive/MyDrive/MIDS/Data 207/207 Final Project/Models/sub_images', exist_ok=True)

# Function to load labels from csv
def load_labels_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df[['center_x', 'center_y', 'width', 'height', 'label']].values.tolist()

# Function to split images using functions defined before
def process_split(split_name, image_dir, bbox_dir, save_path, sub_image_size=64):
    all_sub_images = []
    all_labels = []

    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png') or f.endswith('.jpg')]

    print(f"\nProcessing {split_name} set...")

    for file_name in tqdm(image_files):
        image_path = os.path.join(image_dir, file_name)
        bbox_path = os.path.join(bbox_dir, file_name.replace('.png', '.csv').replace('.jpg', '.csv'))

        if not os.path.exists(bbox_path):
            continue  # Skip if no matching bbox file

        image = np.array(Image.open(image_path).convert("RGB"))
        labels = load_labels_from_csv(bbox_path)

        sub_images, _, sub_image_contains_pedestrian = create_sub_images(image, labels, sub_image_size=sub_image_size)

        for i, sub_img in enumerate(sub_images):
            all_sub_images.append(sub_img)
            all_labels.append(int(sub_image_contains_pedestrian[i]))  # 1 if pedestrian, else 0

    # Convert and save
    all_sub_images = np.array(all_sub_images)
    all_labels = np.array(all_labels)
    np.savez_compressed(save_path, images=all_sub_images, labels=all_labels)
    print(f"Saved {len(all_sub_images)} sub-images to {save_path}")

In [10]:
# Process each split
for split in splits:
    image_dir = os.path.join(BASE_PATH, split, 'images')
    bbox_dir = os.path.join(BASE_PATH, split, 'bboxes')
    save_path = os.path.join(OUTPUT_PATH, f"{split}_data.npz")

    process_split(split, image_dir, bbox_dir, save_path)


Processing train set...


100%|██████████| 477/477 [03:04<00:00,  2.59it/s]


Saved 47700 sub-images to /content/drive/MyDrive/MIDS/Data 207/207 Final Project/Models/sub_images/train_data.npz

Processing val set...


100%|██████████| 239/239 [01:54<00:00,  2.08it/s]


Saved 23900 sub-images to /content/drive/MyDrive/MIDS/Data 207/207 Final Project/Models/sub_images/val_data.npz

Processing test set...


100%|██████████| 239/239 [01:55<00:00,  2.07it/s]


Saved 23900 sub-images to /content/drive/MyDrive/MIDS/Data 207/207 Final Project/Models/sub_images/test_data.npz
